# Creating a Simple Agent with Tracing

In [1]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [2]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

Create a simple Nutrition Assistant Agent

In [3]:
nutrition_agent = Agent(
    name="Nutritio Assistant",
    instructions="""
    You are a helpful assistant giving out nutrition advice.
    You give concise answers
    """
)

Let's execute the Agent:

In [4]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="Nutritio Assistant", ...)
- Final output (str):
    Bananas are very healthy as part of a balanced diet.
    
    Key points:
    - Nutrients: potassium (good for blood pressure), vitamin C, vitamin B6, fiber.
    - Energy: about 90 calories per medium; natural sugars provide quick energy.
    - Benefits: supports heart health, digestion, and satiety.
    
    Notes:
    - Not a complete meal (low in protein/fat); pair with protein/fiber-containing foods.
    - Ripeness changes sugar: underripe has more resistant starch; ripe is sweeter.
    - Kidney disease or very high potassium issues: use caution and check with a clinician.
    
    Simple serving ideas: with yogurt, on cereal, in smoothies, or as a snack.
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


Streaming the answer to the screen, token by token

In [5]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

Bananas are a healthy, convenient fruit. Quick notes:

- Nutrients: potassium, vitamin B6, vitamin C, fiber.
- Benefits: supports heart health, digestion, and energy; naturally sweet with a low fat profile.
- Considerations: contain sugars and carbs; portion size matters for diabetes or calorie goals (1 medium banana ≈ 105 kcal).
- Ripeness: greener = more resistant starch (lower GI); riper = sweeter, higher sugar.
- Overall: great as part of a balanced diet; pair with protein or fats for slower sugar absorption.

_Good Job!_